In [2]:
# Ví dụ về asyncio trong Python

import asyncio
import time
import aiohttp
from typing import List

print("=== CÁC VÍ DỤ VỀ ASYNCIO ===")

# 1. Ví dụ cơ bản về async/await
async def say_hello(name: str, delay: int):
    """Hàm async cơ bản"""
    print(f"Bắt đầu chào {name}...")
    await asyncio.sleep(delay)  # Mô phỏng I/O operation
    print(f"Xin chào {name}!")
    return f"Đã chào {name}"

# Test async function
async def basic_example():
    print("=== 1. VÍ DỤ CƠ BẢN ===")
    result = await say_hello("Alice", 2)
    print(f"Kết quả: {result}")

# Chạy ví dụ cơ bản
await basic_example()


=== CÁC VÍ DỤ VỀ ASYNCIO ===
=== 1. VÍ DỤ CƠ BẢN ===
Bắt đầu chào Alice...
Xin chào Alice!
Kết quả: Đã chào Alice


In [ ]:
# 2. Chạy nhiều task đồng thời với asyncio.gather()
async def fetch_data(name: str, delay: int):
    """Mô phỏng việc lấy dữ liệu từ API"""
    print(f"Bắt đầu fetch dữ liệu cho {name}...")
    await asyncio.sleep(delay)
    result = f"Dữ liệu từ {name}"
    print(f"Hoàn thành fetch: {result}")
    return result

async def concurrent_example():
    print("\n=== 2. CHẠY ĐỒNG THỜI VỚI GATHER ===")
    start_time = time.time()
    
    # Chạy tuần tự (chậm)
    print("Chạy tuần tự:")
    await fetch_data("API_1", 1)
    await fetch_data("API_2", 2)
    await fetch_data("API_3", 1)
    sequential_time = time.time() - start_time
    
    print(f"Thời gian chạy tuần tự: {sequential_time:.2f}s")
    
    # Chạy đồng thời (nhanh)
    print("\nChạy đồng thời:")
    start_time = time.time()
    results = await asyncio.gather(
        fetch_data("API_1", 1),
        fetch_data("API_2", 2),
        fetch_data("API_3", 1)
    )
    concurrent_time = time.time() - start_time
    
    print(f"Kết quả: {results}")
    print(f"Thời gian chạy đồng thời: {concurrent_time:.2f}s")

await concurrent_example()


In [ ]:
# 3. Sử dụng asyncio.create_task() cho background tasks
async def background_task(name: str, duration: int):
    """Task chạy nền"""
    print(f"Background task {name} bắt đầu...")
    await asyncio.sleep(duration)
    print(f"Background task {name} hoàn thành!")
    return f"Task {name} done"

async def main_task():
    """Main task"""
    print("Main task bắt đầu...")
    await asyncio.sleep(1)
    print("Main task hoàn thành!")
    return "Main done"

async def create_task_example():
    print("\n=== 3. CREATE_TASK VÀ BACKGROUND TASKS ===")
    
    # Tạo background tasks
    task1 = asyncio.create_task(background_task("Task1", 3))
    task2 = asyncio.create_task(background_task("Task2", 2))
    
    # Chạy main task trong khi background tasks chạy
    main_result = await main_task()
    
    # Đợi background tasks hoàn thành
    print("Đang đợi background tasks...")
    results = await asyncio.gather(task1, task2)
    
    print(f"Main result: {main_result}")
    print(f"Background results: {results}")

await create_task_example()


In [ ]:
# 4. Xử lý lỗi trong asyncio
async def risky_task(name: str, should_fail: bool = False):
    """Task có thể gây lỗi"""
    print(f"Task {name} bắt đầu...")
    await asyncio.sleep(1)
    
    if should_fail:
        raise ValueError(f"Task {name} gặp lỗi!")
    
    print(f"Task {name} thành công!")
    return f"Success: {name}"

async def error_handling_example():
    print("\n=== 4. XỬ LÝ LỖI TRONG ASYNCIO ===")
    
    # Xử lý lỗi với try-except
    try:
        result = await risky_task("Safe Task", should_fail=False)
        print(f"Kết quả: {result}")
    except ValueError as e:
        print(f"Lỗi: {e}")
    
    # Xử lý lỗi với asyncio.gather và return_exceptions=True
    print("\nXử lý lỗi với gather:")
    results = await asyncio.gather(
        risky_task("Task1", should_fail=False),
        risky_task("Task2", should_fail=True),
        risky_task("Task3", should_fail=False),
        return_exceptions=True
    )
    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Task {i+1} gặp lỗi: {result}")
        else:
            print(f"Task {i+1} thành công: {result}")

await error_handling_example()


In [ ]:
# 5. Async Context Managers và Async Iterators
class AsyncResource:
    """Async context manager example"""
    def __init__(self, name: str):
        self.name = name
        self.is_open = False
    
    async def __aenter__(self):
        print(f"Mở resource: {self.name}")
        await asyncio.sleep(0.5)  # Mô phỏng việc mở resource
        self.is_open = True
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print(f"Đóng resource: {self.name}")
        await asyncio.sleep(0.5)  # Mô phỏng việc đóng resource
        self.is_open = False
    
    async def do_work(self):
        if self.is_open:
            print(f"Làm việc với {self.name}")
            await asyncio.sleep(1)
            return f"Work done with {self.name}"
        else:
            raise RuntimeError(f"Resource {self.name} chưa được mở!")

class AsyncCounter:
    """Async iterator example"""
    def __init__(self, max_count: int):
        self.max_count = max_count
        self.count = 0
    
    def __aiter__(self):
        return self
    
    async def __anext__(self):
        if self.count >= self.max_count:
            raise StopAsyncIteration
        
        await asyncio.sleep(0.5)  # Mô phỏng async operation
        self.count += 1
        return self.count

async def async_patterns_example():
    print("\n=== 5. ASYNC CONTEXT MANAGERS VÀ ITERATORS ===")
    
    # Async context manager
    print("Async Context Manager:")
    async with AsyncResource("Database") as db:
        result = await db.do_work()
        print(f"Kết quả: {result}")
    
    # Async iterator
    print("\nAsync Iterator:")
    async for number in AsyncCounter(5):
        print(f"Count: {number}")

await async_patterns_example()


In [ ]:
# 6. Ví dụ thực tế: Gọi nhiều API đồng thời
import json
try:
    import aiohttp
    AIOHTTP_AVAILABLE = True
except ImportError:
    AIOHTTP_AVAILABLE = False
    print("Cần cài đặt aiohttp: pip install aiohttp")

# Mô phỏng HTTP requests nếu không có aiohttp
async def mock_http_request(url: str, delay: int = 1):
    """Mô phỏng HTTP request"""
    print(f"Gọi API: {url}")
    await asyncio.sleep(delay)
    return {
        "url": url,
        "status": 200,
        "data": f"Response from {url}"
    }

async def fetch_multiple_apis():
    """Gọi nhiều API đồng thời"""
    print("\n=== 6. VÍ DỤ THỰC TẾ: GỌI NHIỀU API ===")
    
    urls = [
        "https://api.example1.com/data",
        "https://api.example2.com/users", 
        "https://api.example3.com/products",
        "https://api.example4.com/orders"
    ]
    
    start_time = time.time()
    
    # Gọi tất cả API đồng thời
    print("Gọi tất cả API đồng thời...")
    tasks = [mock_http_request(url, delay=2) for url in urls]
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    
    print(f"\nKết quả:")
    for result in results:
        print(f"- {result['url']}: {result['status']} - {result['data']}")
    
    print(f"\nThời gian thực hiện: {end_time - start_time:.2f}s")
    print(f"Nếu gọi tuần tự sẽ mất: {len(urls) * 2}s")

await fetch_multiple_apis()


In [ ]:
# 7. Asyncio Best Practices và Patterns
import asyncio
from asyncio import Queue

# Producer-Consumer pattern
async def producer(queue: Queue, producer_id: int):
    """Sản xuất dữ liệu"""
    for i in range(5):
        item = f"Item-{producer_id}-{i}"
        await queue.put(item)
        print(f"Producer {producer_id} tạo: {item}")
        await asyncio.sleep(0.5)
    
    await queue.put(None)  # Signal completion

async def consumer(queue: Queue, consumer_id: int):
    """Tiêu thụ dữ liệu"""
    while True:
        item = await queue.get()
        if item is None:
            break
        
        print(f"Consumer {consumer_id} xử lý: {item}")
        await asyncio.sleep(1)  # Mô phỏng processing
        queue.task_done()

async def producer_consumer_example():
    print("\n=== 7. PRODUCER-CONSUMER PATTERN ===")
    
    # Tạo queue với kích thước giới hạn
    queue = asyncio.Queue(maxsize=3)
    
    # Tạo producers và consumers
    producers = [asyncio.create_task(producer(queue, i)) for i in range(2)]
    consumers = [asyncio.create_task(consumer(queue, i)) for i in range(2)]
    
    # Đợi producers hoàn thành
    await asyncio.gather(*producers)
    
    # Đợi tất cả items được xử lý
    await queue.join()
    
    # Dừng consumers
    for _ in consumers:
        await queue.put(None)
    
    await asyncio.gather(*consumers)
    print("Producer-Consumer example hoàn thành!")

# Rate limiting pattern
async def rate_limited_task(semaphore: asyncio.Semaphore, task_id: int):
    """Task với rate limiting"""
    async with semaphore:
        print(f"Task {task_id} bắt đầu...")
        await asyncio.sleep(2)
        print(f"Task {task_id} hoàn thành!")
        return f"Result {task_id}"

async def rate_limiting_example():
    print("\n=== 8. RATE LIMITING ===")
    
    # Chỉ cho phép 3 task chạy đồng thời
    semaphore = asyncio.Semaphore(3)
    
    # Tạo 10 tasks
    tasks = [rate_limited_task(semaphore, i) for i in range(10)]
    
    start_time = time.time()
    results = await asyncio.gather(*tasks)
    end_time = time.time()
    
    print(f"Hoàn thành {len(tasks)} tasks trong {end_time - start_time:.2f}s")
    print(f"Kết quả: {results[:3]}...")  # Show first 3 results

print("=== CHẠY CÁC VÍ DỤ NÂNG CAO ===")
await producer_consumer_example()
await rate_limiting_example()

print("\n=== TỔNG KẾT ===")
print("✅ Đã học các pattern quan trọng của asyncio:")
print("1. Async/await cơ bản")
print("2. Chạy đồng thời với gather()")
print("3. Background tasks với create_task()")
print("4. Xử lý lỗi trong async code")
print("5. Async context managers và iterators")
print("6. Gọi API đồng thời")
print("7. Producer-Consumer pattern")
print("8. Rate limiting với Semaphore")
